<img src="./images/banner.png" width="100%" align="left" />

<table style="float:right;">
    <tr>
        <td>                      
            <div style="text-align: right"><a href="https://alandavies.netlify.com" target="_blank">Dr Alan Davies</a></div>
            <div style="text-align: right">Senior Lecturer Health Data Science</div>
            <div style="text-align: right">University of Manchester</div>
         </td>
         <td>
             <img src="./images/alan.png" width="30%" />
         </td>
     </tr>
</table>

# 4.0 Data visualisation for geo-spacial data in R
****

#### About this Notebook
This notebook introduces the concepts of <code>spacial</code> data and how we can use maps and other geographical information to create data plots.

<div class="alert alert-block alert-warning"><b>Learning Objectives:</b> 
<br/> At the end of this notebook you will be able to:
    
    
- LO9 Practice plotting data using appropriate visualisation types, including static and dynamic (interactive) forms of visualisation using modern graph libraries in languages like R and Python

- LO2 Discuss the best design practices for presenting data with visualisations

- LO5	Recognise and select appropriate visualisations to communicate to different audiences with differing levels of technical and clinical understanding   
    
</div> 

<a id="top"></a>

<b>Table of contents</b><br>

4.1 [Geographical regions](#types)

4.2 [Point map](#point)

Another way to visualise data is based on geographical properties. This is useful when the location is important. Examples could include things like number of cases of an infectious disease per county or region, or the budget expenditure for groups of GP (General Practitioner) practices known as primary care networks (PCNs). 

<a id="types"></a>
#### 4.1 Geographical regions

How a region or country is broken down into boundaries may not be a simple task. In the case of England for example, we could choose to split the regions by <code>post code</code> (zip code), <code>output area</code> (based on census data), <code>local government districts/wards</code>, <code>local education authorities</code> and so on. As we work in healthcare we are usually interested in the various health based boundaries. As a data scientist, you may also link these data to other data sets such as health inequalities etc.

<div class="alert alert-success">
<b>Note:</b> If you are interested in a detailed overview of the different boundary types, check out this guide from the Office of National Statistics: <a href="https://geoportal.statistics.gov.uk/documents/a-beginners-guide-to-uk-geography-2021-v1-0-1/explore" target="_blank">ONS: A beginners guide to UK geography</a>.
</div>

A common type of visualisation used for map data is the <code>choropleth</code>. This plot is a thematic map that highlights (with colours/patterns) a boundaried geographical area based on statistical properties. Take a look at an example below from the United States Centers for Disease Control and Prevention (CDC) around opioid dispensing rates from the years 2006 to 2020.

<img src="./images/2006-2020-map.gif" width="100%" align="left" />

<div class="alert alert-block alert-info">
<b>Task 1:</b>
<br> 
    1. What sort of information do plots like this convey?<br>
    2. How might the plot above be used?
</div>

<strong>1.</strong> They allow us to see a proportion or quantity per region. In this case opioid dispensing rates per 100 people. The heatmap colours can be used to highlight areas of interest (e.g. low or high dispensing rates). As the above plot is also animated, we can see changes over time showing the increase in opioid dispensing across the country followed by a general decrease. <br>
<strong>2.</strong> Such a plot could be used to target resources or interventions in areas with the most need. Comparison with earlier plots could also be seen if there are any changes in the various areas over time.

There are many different Geographic Information Systems (GIS). These are systems that can analyse and display information that is geographically referenced. There are many different geographical items that may be of interest such as the water ways, roads, mountains, boundary areas and so on. This is summarised in the image below (from <a href="https://www.usgs.gov/faqs/what-geographic-information-system-gis" target="_blank">https://www.usgs.gov/faqs/what-geographic-information-system-gis</a>). This is why it's not as straight forward to produce map based visualisations as it is other types.

<img src="./images/gis.jpg" width="35%" align="left" />

Getting this sort of data is country and region specific. In the UK, a lot of this data can be found via the Office for National Statistics (ONS) Open Geography Portal (<a href="https://geoportal.statistics.gov.uk/search?collection=Dataset&sort=name&tags=all(BDY_HLT)" target="_blank">https://geoportal.statistics.gov.uk/search?collection=Dataset&sort=name&tags=all(BDY_HLT)</a>). Another useful place to find map data is via Ordnance Surveys OpenData support (<a href="https://www.ordnancesurvey.co.uk/business-government/tools-support/open-data-support" target="_blank">https://www.ordnancesurvey.co.uk/business-government/tools-support/open-data-support</a>).

Map data for generating boundaries and other geographical features comes in many forms. One of the common forms is that of 
<code>geoJSON</code>. JSON (JavaScript Object Notation) involves nested objects (<code>{}</code>), arrays/lists (<code>[]</code>) and key/value pairs separated by colons e.g. <code>some_key: some_value</code>. Below is an example from <a href="https://martinjc.github.io/UK-GeoJSON/" target="_blank">https://martinjc.github.io/UK-GeoJSON/</a>.

<code>
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [125.6, 10.1]
  },
  "properties": {
    "name": "Dinagat Islands"
  }
}
</code>

We will create a choropleth to show some data using the geoJSON format. We start by loading the required R libraries:

In [27]:
library(tidyverse)
library(jsonlite)

There are two files that we need to read in, the first is the UK geo data that defined the boundaries we are interested in. In this case it's the local authority areas. We can load this in as <code>json</code> file.

In [28]:
UK_geodata <- fromJSON("./data/geodata.json")

Next we will load a dataset on the estimates of opiate and crack cocaine use prevalence: 2016 to 2017 from <a href="https://www.gov.uk/government/publications/opiate-and-crack-cocaine-use-prevalence-estimates-for-local-populations" target="_blank">Public Health England</a>. We extracted some data from this in the form of a CSV file which we will load in the standard way using the <code>pandas</code> package.

In [29]:
df <- read_csv("./data/OpiateCrackUsePrevalenceEstimates2016to2017.csv")

Parsed with column specification:
cols(
  Region = col_character(),
  `Local authority` = col_character(),
  Opiates_2010_11 = col_character(),
  Opiates_2011_12 = col_character(),
  `Opiates_2012-13` = col_double(),
  Opiates_2013_14 = col_double(),
  Opiates_2014_15 = col_double(),
  Opiates_2016_17 = col_double()
)


We can use the <code>head</code> function to display the first few rows of data. You can see we have various regions and local authorities followed by the prevalence of opiate use over several years (from 2010 to 2017).

In [30]:
head(df, 5)

Region,Local authority,Opiates_2010_11,Opiates_2011_12,Opiates_2012-13,Opiates_2013_14,Opiates_2014_15,Opiates_2016_17
East Midlands,Derby,1948,2094,2004,1974,1989,1826
East Midlands,Derbyshire,3208,3331,3259,3537,3481,3275
East Midlands,Leicester,2439,2533,2429,2548,2298,2327
East Midlands,Leicestershire,1848,1665,1829,1976,2285,2351
East Midlands,Lincolnshire,2710,2762,2518,3044,2929,3241


We can check the structure of the dataframe and it's properties with the <code>summary</code> function:

In [31]:
summary(df)

    Region          Local authority    Opiates_2010_11    Opiates_2011_12   
 Length:153         Length:153         Length:153         Length:153        
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                                                                            
 Opiates_2012-13 Opiates_2013_14 Opiates_2014_15 Opiates_2016_17
 Min.   :  20    Min.   :  25    Min.   :  23    Min.   :  14   
 1st Qu.: 956    1st Qu.: 927    1st Qu.: 936    1st Qu.: 996   
 Median :1394    Median :1475    Median :1411    Median :1500   
 Mean   :1669    Mean   :1720    Mean   :1714    Mean   :1738   
 3rd Qu.:2053    3rd Qu.:2153    3rd Qu.:2098    3rd Qu.:2154   
 Max.   :8129    Max.   :8700    Max.   :8234    Max.   :8779   

For this simple example we will extract a subset of data for the year 2010 to 2011 and then view the first records to check that this operation was successful. 

In [32]:
year2010to2011 <- df %>% select(c("Region", "Local authority", "Opiates_2010_11"))

In [33]:
head(year2010to2011, 5)

Region,Local authority,Opiates_2010_11
East Midlands,Derby,1948
East Midlands,Derbyshire,3208
East Midlands,Leicester,2439
East Midlands,Leicestershire,1848
East Midlands,Lincolnshire,2710


Next we will turn our attention back to geo data which is quite different in structure. In JSON, there are two main parts: keys and values and together they make a key/value pair. In the geodata JSON file, we can use R to output the key's for the various geo <code>features</code>.

In [34]:
keys <- names(UK_geodata$features)
print(keys)

[1] "type"       "geometry"   "properties" "id"        


We can explore the data structure further by taking a look at the <code>properties</code> for the 1st item. You can see an example of the sort of information represented, you can see the name is <code>Middlesbrough</code> and other information, such as the longitude and latitude.

# cant get this to look up the correct feature properties - keeps coming back null or issue with atomic vectors

In [39]:
#in Python: UK_geodata['features'][1]['properties']

library(jsonlite)
second_feature_properties <- UK_geodata$features[[2]]$properties
print(second_feature_properties)

NULL


Next we need to set the value for the features id to be the name of the local authority area. We use this to map the two datasets together. We use a loop to repeat this operation for each feature in the dataset.

In [42]:
# python version:
#for feature in UK_geodata['features']:
#    feature['id'] = feature['properties']['name'] 

#iterate through features and match up id with properties name:
for (feature in UK_geodata$features)
    (feature$id == feature$properties$name)  

ERROR: Error: $ operator is invalid for atomic vectors


Now we are ready to create the choropleth. We can use the <code>choropleth</code> function from the <code>plotly</code> library. We pass in the data set and map the <code>locations</code> to the appropriate column (<code>Local authority</code>). Finally we also pass in the geoJSON file.

In [ ]:
# px.choropleth(year2010to2011, locations="Local authority", geojson=UK_geodata)





You can zoom in/out with your mouse wheel and click (hold) and drag to move the map around. When you hover over the regions, you can see the name of the various region. It doesn't look very impressive however. We can use something like <code>Google maps</code> or <code>Mapbox</code> to add more detail to our maps. We use the function <code>choropleth_mapbox</code> in this instance to add some more detail to the map. We also use colour to show the different values and choose a start level of zoom and long/lat for the initial position. This could be used to zoom in on a region in a specific country for example.

In [ ]:
px.choropleth_mapbox(year2010to2011, 
                     locations="Local authority", 
                     geojson=UK_geodata, 
                     color="Opiates_2010_11", 
                     mapbox_style="carto-positron",
                     center={'lat': 55, 'lon': 3.4},
                     zoom=3,
                     opacity=0.3)

Again, you can zoom in/out and move the map around. This version looks a lot better than the first version.

<div class="alert alert-danger">
<b>Note:</b> There are many different classification systems that can be used to split data into the various different areas. The method chosen can radically alter the appearance (and thus the interpretation) of the map data. Some of the commonly used methods include:
- Equal count (quantile): same amount of map areas allocated to each class
- Equal interval: evenly spaced gaps between class breaks
- Jenks natural breaks: An optimisation method that uses clusters to define breaks. This was the method used on the very first choropleth we showed you in this notebook from the CDC.
</div>

***

Here is some data from some applicants for one our courses.

In [45]:
# applicant_data = pd.read_csv("./data/applicant_data.csv")
# applicant_data.head(5)

applicant_data <- read_csv("./data/applicant_data.csv")
head(applicant_data, 5)

Parsed with column specification:
cols(
  Sex = col_character(),
  Age = col_double(),
  Location = col_character(),
  Region = col_character()
)


Sex,Age,Location,Region
Male,44,UK,Suffolk
Female,30,UK,Greater London
Male,32,UK,Oxfordshire
Female,43,UK,Somerset
Female,38,UK,London


<div class="alert alert-block alert-info">
<b>Task 2:</b>
<br> 
    Create a choropleth to show how many applicants we had from each region. You can load in the same geoJSON file that was used previously.
</div>

In [ ]:
# # Load in the geo data JSON file
# geo_data = json.load(open("./data/geodata.json", "r"))

# # Remap the id 
# for feature in geo_data['features']:
#     feature['id'] = feature['properties']['name']
    
# # We need to group (aggregate) the applicants by region
# region_count = applicant_data.groupby(['Region']).size().to_frame(name = "Value").reset_index()

# # Create the choropleth
# px.choropleth_mapbox(region_count, 
#                      locations="Region", 
#                      geojson=geo_data, 
#                      color="Value", 
#                      mapbox_style="carto-positron",
#                      center={'lat': 55, 'lon': 3.4},
#                      zoom=3,
#                      opacity=0.3)

<a id="point"></a>
#### 4.2 Point map

Another type of spatial plot that is commonly used is a point map. This simply places points on a map. This can be used to see the number of something in an area. We have used this in the past to gather research data on mobile health apps to see where our users are geographically located. To see an example of this we will use some data on the geo-locations of GP practices in Manchester and plot them on the map to see where the main clusters of practices are located throughout the city.

<div class="alert alert-danger">
<b>Note:</b> Geo-location data can lead to the identification of individuals if used with other data. when collecting this type of data for research, collect the highest level possible. For example you might want to know how many people downloaded your intervention per post code or area rather then their exact location. Sometimes an artificial offset is added to alter the accuracy of geo data or it is only recorded if more than a certain number of users are in a particular area. You can see an example of this in an app designed to collect allergy information from the general public <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6381818/" target="_blank">here</a> where they approximated the number of participants in each postcode. 
</div>

This time we will use the <code>ggplot2</code> library.

In [46]:
#import matplotlib.pyplot as plt

library(ggplot2)

Next we can load in a png image of the map for the background. This shows the main Manchester city area. This was exported from <a href="https://www.openstreetmap.org/" target="_blank">OpenStreetMap</a>. This lets a user select a region and export it. It also provides the longitude and latitude information for the selected area.

In [ ]:
#map_img = plt.imread("./images/map.png")

Next we load in the dataset containing details of the various practices based in Manchester. We have the post code and latitude/longitude information here. 

In [47]:
# practices_df = pd.read_csv("./data/practices.csv")
# practices_df.head(5)

practices_df <- read_csv("./data/practices.csv")
head(practices_df, 5)

Parsed with column specification:
cols(
  code = col_character(),
  practice = col_character(),
  address = col_character(),
  area = col_character(),
  county = col_character(),
  postcode = col_character(),
  lat = col_double(),
  long = col_double()
)


code,practice,address,area,county,postcode,lat,long
N81001,AUDLEM MEDICAL PRACTICE,"16 CHESHIRE ST, AUDLEM",CREWE,CHESHIRE,CW3 0AH,52.98962,-2.50872
N81002,KENMORE MEDICAL CENTRE,60-62 ALDERLEY ROAD,WILMSLOW,CHESHIRE,SK9 1PA,53.32308,-2.23522
N81005,HELSBY HEALTH CENTRE,LOWER ROBIN HOOD LANE,"HELSBY, WARRINGTON",CHESHIRE,WA6 0BW,53.27047,-2.77052
N81006,BUNBURY MEDICAL PRACTICE,"VICARAGE LANE, BUNBURY",TARPORLEY,CHESHIRE,CW6 9PE,53.11683,-2.64938
N81007,HOLES LANE MEDICAL CENTRE,"THE SURGERY,28 HOLES LANE","WOOLSTON, WARRINGTON",CHESHIRE,WA1 4NE,53.39983,-2.54511


<div class="alert alert-success">
<b>Note:</b> If you need the latitude/longitude points and only have post codes, there are plenty of tools that you can use to convert them such as <a href="https://www.freemaptools.com/convert-uk-postcode-to-lat-lng.htm" target="_blank">https://www.freemaptools.com/convert-uk-postcode-to-lat-lng.htm</a>.
</div>

Next we need to create a bounding box (a rectangle/square) that is made up of the maximum and minimum latitude and longitude points. We can use the <code>min()</code> and <code>max()</code> functions for this.

In [49]:
# BBox = (practices_df.long.min(), practices_df.long.max(), practices_df.lat.min(), practices_df.lat.max())
# BBox



Finally we can put this all together using a scatter plot with the latitude/longitude locations of the practices setting the colour (<code>c='r'</code>), transparency (<code>alpha=0.8</code>) and size (<code>s=12</code>). We set the x and y axis limits to the dimensions of the bounding box <code>BBox</code> tuple that we created eralier. 

In [ ]:
# fig, ax = plt.subplots(figsize = (12, 10))
# ax.scatter(practices_df.long, practices_df.lat, zorder=1, alpha=0.8, c='r', s=12);
# ax.set_title('GP practices in Manchester (UK)')
# ax.set_xlim(BBox[0], BBox[1])
# ax.set_ylim(BBox[2], BBox[3])
# ax.imshow(map_img, zorder=0, extent = BBox, aspect='equal')

There are of course other types of spacial plots that are used such as the <code>proportional symbol map</code>. This can be used to show magnitude. For example we could make the circles in the map above larger or smaller to represent the number of patients that the various practices process. 

As you can see, generating plots using geographical data can be more complex than other forms of visualisations and requires a knowledge of boundary systems and often the use of various APIs (Application Programming Interfaces) to combine the different features into something that works well. We have only touched on some of the tools and techniques that can be used to visualise these sorts of data.

#### Feedback
Please rate this notebook by clicking on the image below (it will open up a new window where you can select one of the three icons)
 <a href="https://www.qualtrics.manchester.ac.uk/jfe/form/SV_d0xXBbVTXslv9ps" target="_blank"><img src="./images/sentiment-check-icons.png" width="80%" /></a>

------

##### About this Notebook
<br>
<i>Notebook created by <strong>Dr. Alan Davies</strong>.
    
Publish date: July 2023<br>
Review date: July 2024</i>

## Notes: